# Analise de sentimento

baseado no post do medium de: https://medium.com/@alegeorgelustosa/an%C3%A1lise-de-sentimentos-em-python-2a7d04a836e0

Para esse exemplo foi uitlizado uma base de textos, já classificados, obtidos a partir do post original do medium. A partir dessa base vamos treinar um modelo bayesiano que será capaz de predizer o sentimento com as possíveis labels: Alegria, Nojo, Medo, Raíva, Surpresa e Tristeza.

## Imports

Vamos carregar as bibliotecas necessárias para executar o exemplo.

In [1]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

##  Read csv file

Vamos carregar o dataset contendo todos os exemplos já classificados que serão utilizados para gerar um modelo capaz de predizer um novo rotulo a uma frase.

In [2]:
df = pd.read_csv('./datasets/sentimentos.csv')

## View Data

Aqui vamos visualizar 10 exemplos de textos e sua classificação.

In [3]:
df.sample(10)

,Texto,Sentimento
115,que banheiro porco,nojo
313,a noite e muito perigosa,medo
335,tirei ferias e fui despedido,raiva
320,seu políticos usam suas forças para afugentar ...,medo
751,"nossa que alto aqui, eu nao gosto de altura",medo
42,estou cativada pelo seu olhar,alegria
533,essas pessoas são esplêndida,surpresa
602,as pessoas não gostam do meu jeito,tristeza
149,a indisposição me atacou hoje,nojo
686,você me completa,alegria


In [4]:
df['Sentimento'].value_counts()

alegria     162
medo        156
raiva       152
tristeza    149
nojo        143
surpresa    141
Name: Sentimento, dtype: int64

Vamos carregar apenas 140 frases de cada tipo de sentimento a fim de equalizar os exemplos para evitar assim um favorecimento entre os tipos de exemplos.

In [5]:
df_alegria = df.loc[df.Sentimento == 'alegria'][0:140]
df_medo = df.loc[df.Sentimento == 'medo'][0:140]
df_raiva = df.loc[df.Sentimento == 'raiva'][0:140]
df_tristeza = df.loc[df.Sentimento == 'tristeza'][0:140]
df_nojo = df.loc[df.Sentimento == 'nojo'][0:140]
df_surpresa = df.loc[df.Sentimento == 'surpresa'][0:140]

In [6]:
df = pd.concat([df_alegria, df_medo, df_raiva, df_tristeza, df_nojo, df_surpresa], ignore_index=True, sort=True)

In [7]:
df.Sentimento.value_counts()

raiva       140
medo        140
alegria     140
tristeza    140
nojo        140
surpresa    140
Name: Sentimento, dtype: int64

## Load stop words 

Stop Words são palavras que não ajudam muito o modelo a conseguir realizar uma predição correta, elas não trazem um grande valor para o processo de predição por isso podem ser removidas.

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stop_words = nltk.corpus.stopwords.words('portuguese')
np.transpose(stop_words[0:20])

array(['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para',
       'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais'],
      dtype='<U4')

## Stemming: Removing sulfix and prefix of word

Stemming é um pré-processo para remoção das variações de determinada palavra, trazendo ela a seu radical.

In [10]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [11]:
def apply_stemmer(text: str):
    stemmer = nltk.stem.RSLPStemmer()
    texts = []
    texts = [str(stemmer.stem(word)) for word in text.split() if word not in stop_words]
    return tuple(texts)

In [12]:
apply_stemmer('Ola admirável mundo novo, eu programei você!! Eu sou um programador')

('ola', 'admir', 'mund', 'novo,', 'program', 'você!!', 'eu', 'program')

In [13]:
df['Palavras'] = df['Texto'].apply(apply_stemmer)
df.sample(10)

,Sentimento,Texto,Palavras
20,alegria,o dia esta muito bonito,"(dia, bonit)"
372,raiva,o que você tem com isso?,"(isso?,)"
663,nojo,abominavelmente convencido,"(abomina, convenc)"
564,nojo,bebi demais e preciso vomitar,"(beb, demal, precis, vomit)"
113,alegria,não precisei pagar o ingresso,"(precis, pag, ingress)"
821,surpresa,ah! o absoluto do imaginário,"(ah!, absolut, imagin)"
330,raiva,meus pais não consentiram nosso casamento,"(pal, consent, cas)"
560,nojo,você e abominável,"(abomin,)"
60,alegria,e um enorme prazer ter você em nossa equipe,"(enorm, praz, ter, equip)"
649,nojo,"não sei como e a vida de um patife, mais a de ...","(sei, vid, patife,, hom, honest, abomin)"


## Getting words

Aqui vamos obter todas as palavras já processadas de cada frase, a fim de criar um saco de palavras, ou BagOfWords como é conhecido.

In [14]:
words = []
for index, item in df.iterrows():
    words.extend(item.Palavras)

In [15]:
len(words)

2627

## Genereting the frequency distribution

Vamos gerar aqui uma distribuição de frequencia a fim de identificar quais são as palavras mais relevantes

In [16]:
frequency = nltk.FreqDist(words)
frequency.most_common(10)

[('vou', 30),
 ('nao', 30),
 ('tão', 21),
 ('quer', 21),
 ('tod', 20),
 ('tud', 18),
 ('ser', 18),
 ('fic', 17),
 ('sint', 17),
 ('pod', 14)]

## Getting unique words

Com base nas palavras obtidas, vamos pegar apenas as palavras que não se repetem para utiliza-las como base para criar uma matriz binaria, a fim de identificar quais palavras estão disponiveis em cada frase.

In [17]:
unique_words = frequency.keys()
print(len(unique_words))

1245


In [18]:
def extract_words_to_text(text):
    doc = None
    if(type(text) == type(str)):
        doc = text.split()
    else:
        doc = set(text)
        
    features = {}
    for word in unique_words:
        features['%s' % word] = (word in doc)
    return features

In [19]:
def extract_data(df: pd.DataFrame):
    itens = []
    for i, row in df.iterrows():
        field = (tuple(row['Palavras']), row['Sentimento'])
        itens.append(field)
    return itens

In [20]:
extract_data(df)[0:5]

[(('trabalh', 'agrad'), 'alegria'),
 (('gost', 'fic', 'aconcheg'), 'alegria'),
 (('fiz', 'ades', 'curs', 'hoj', 'porqu', 'gost'), 'alegria'),
 (('admir', 'muit'), 'alegria'),
 (('ador',), 'alegria')]

## Splitting the data frame into train and test

Com a matriz binaria montada e classficada, vamos dividir o dataset em dois blocos, que serão utilizados para treinar o modelo e testar o modelo.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df[['Texto', 'Palavras']], df['Sentimento'], train_size=0.85, random_state=42)

In [22]:
y_train.value_counts()

medo        124
raiva       123
surpresa    120
nojo        118
tristeza    115
alegria     114
Name: Sentimento, dtype: int64

In [23]:
y_test.value_counts()

alegria     26
tristeza    25
nojo        22
surpresa    20
raiva       17
medo        16
Name: Sentimento, dtype: int64

In [24]:
X_train['Sentimento'] = y_train

## Train model

Vamos agora aplicar a base de treinamento a extração das palavras para criar uma matriz binaria que será utilizada para o classificar aprender os padrões

In [25]:
data = nltk.classify.apply_features(extract_words_to_text, extract_data(X_train), labeled=True)

In [26]:
classificador = nltk.NaiveBayesClassifier.train(data)

In [27]:
classificador.labels()

['raiva', 'tristeza', 'medo', 'surpresa', 'alegria', 'nojo']

In [28]:
classificador.show_most_informative_features(10)

Most Informative Features
                     med = True             medo : alegri =      7.1 : 1.0
                     vou = True            raiva : surpre =      6.8 : 1.0
                 acredit = True           surpre : triste =      6.7 : 1.0
                   terri = True             nojo : triste =      4.9 : 1.0
                      am = True           alegri : surpre =      4.6 : 1.0
                     tão = True           surpre : raiva  =      4.4 : 1.0
                     nao = True             medo : nojo   =      4.4 : 1.0
                     ser = True             medo : raiva  =      4.3 : 1.0
                    quer = True            raiva : surpre =      4.2 : 1.0
                    real = True           surpre : triste =      4.2 : 1.0


## Evalueting model

Com base no modelo aprendido com a base de treinamento podemos avaliar o modelo e verificar a sua acurácia na predição dos dados já rotulados.

In [29]:
print(f'Acerto: {round(nltk.classify.accuracy(classificador, data) * 100, 2)}%')

Acerto: 96.08%


In [30]:
def error_metric(data) -> []:
    erros = []
    for (words, target) in data:
        result = classificador.classify(words)
        if result != target:
            erros.append((target, result, words))
    print(f'Erros: {(round(len(erros) / len(data) * 100,2))} %')

In [31]:
error_metric(data)

Erros: 3.92 %


In [32]:
from nltk.metrics import ConfusionMatrix

def view_confusion_matrix(data):
    y = []
    pred = []
    for (word, target) in data:
        result = classificador.classify(word)
        y.append(target)
        pred.append(result)

    matriz = ConfusionMatrix(y, pred)
    print(matriz)

A matriz de confução nos mostra como cada registro foi classificado.

In [33]:
view_confusion_matrix(data)

         |                   s   t |
         |   a               u   r |
         |   l               r   i |
         |   e           r   p   s |
         |   g   m   n   a   r   t |
         |   r   e   o   i   e   e |
         |   i   d   j   v   s   z |
         |   a   o   o   a   a   a |
---------+-------------------------+
 alegria |<111>  .   .   1   2   . |
    medo |   1<116>  2   3   1   1 |
    nojo |   .   .<114>  2   1   1 |
   raiva |   .   2   1<120>  .   . |
surpresa |   .   1   .   .<119>  . |
tristeza |   2   3   .   3   1<106>|
---------+-------------------------+
(row = reference; col = test)



## Teste model

Vamos agora avaliar o modelo criado com a base de testes a fim de validar o quao bem esse modelo seria se estivesse em um ambiente produtivo.

In [34]:
X_test['Sentimento'] = y_test
data_test = nltk.classify.apply_features(extract_words_to_text, extract_data(X_test), labeled=True)

In [35]:
print(f'Acerto: {round(nltk.classify.accuracy(classificador, data_test) * 100, 2)}%')

Acerto: 44.44%


In [36]:
error_metric(data_test)

Erros: 55.56 %


Matriz de confusão sobre os dados que foram utilizados para o teste

In [37]:
view_confusion_matrix(data_test)

         |              s  t |
         |  a           u  r |
         |  l           r  i |
         |  e        r  p  s |
         |  g  m  n  a  r  t |
         |  r  e  o  i  e  e |
         |  i  d  j  v  s  z |
         |  a  o  o  a  a  a |
---------+-------------------+
 alegria |<15> 5  .  2  2  2 |
    medo |  . <6> .  2  3  5 |
    nojo |  5  1 <9> 2  2  3 |
   raiva |  2  3  2 <8> 2  . |
surpresa |  6  .  1  4 <8> 1 |
tristeza |  4  2  1  6  2<10>|
---------+-------------------+
(row = reference; col = test)



Vamos criar um bloco de código que será utilizado para predizer agora as novas frases

In [38]:
def predict_test(text):
    text = apply_stemmer(text)
    novo = extract_words_to_text(text)

    result = classificador.classify(novo)
    print('Predicação: %s' % result)

    print()
    distribuicao = classificador.prob_classify(novo)
    for clas in distribuicao.samples():
        print('%s: %f' % (clas, distribuicao.prob(clas)))

In [39]:
test = 'Eu vou ser pai!!!'
predict_test(test)

Predicação: medo

raiva: 0.210226
tristeza: 0.044344
medo: 0.525327
surpresa: 0.048977
alegria: 0.024613
nojo: 0.146512


In [40]:
test = 'Fui assaltado ontem'
predict_test(test)

Predicação: surpresa

raiva: 0.273349
tristeza: 0.085567
medo: 0.168748
surpresa: 0.286206
alegria: 0.101733
nojo: 0.084396


In [41]:
test = 'Ganhei na mega sena'
predict_test(test)

Predicação: tristeza

raiva: 0.137687
tristeza: 0.368931
medo: 0.085693
surpresa: 0.140632
alegria: 0.143689
nojo: 0.123368


In [42]:
test = 'Estou namorando uma gata linda'
predict_test(test)

Predicação: alegria

raiva: 0.012283
tristeza: 0.037629
medo: 0.022750
surpresa: 0.120597
alegria: 0.770575
nojo: 0.036167
